<a href="https://colab.research.google.com/github/ashimakeshava/NMA_marmots/blob/master/Behavioral_DF_Paula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Data retrieval
import os, requests
import numpy as np
import pandas as pd
from pandas import DataFrame

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [3]:
#@title Data loading


alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]
print(dat.keys())


dict_keys(['spks', 'wheel', 'pupil', 'lfp', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'brain_area_lfp', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'waveform_w', 'waveform_u', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'lfp_passive', 'pupil_passive', 'wheel_passive'])


alldat contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each dat = alldat[k], you have the following fields:


*   dat['mouse_name']: mouse name
*   dat['date_exp']: when a session was performed
*   dat['spks']: neurons by trials by time bins.
*   dat['brain_area']: brain area for each neuron recorded.
*   dat['contrast_right']: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
*   dat['contrast_left']: contrast level for left stimulus.
*   dat['gocue']: when the go cue sound was played.
*   dat['response_times']: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.
*   dat['response']: which side the response was (-1, 0, 1). When the right-side stimulus had higher contrast, the correct choice was -1. 0 is a no go response.
*   dat['feedback_time']: when feedback was provided.
*   dat['feedback_type']: if the feedback was positive (+1, reward) or negative (-1, white noise burst).
*   dat['wheel']: exact position of the wheel that the mice uses to make a response, binned at 10ms.
*   dat['pupil']: pupil area (noisy, because pupil is very small) + pupil horizontal and vertical position.
*   dat['lfp']: recording of the local field potential in each brain area from this experiment, binned at 10ms.
*   dat['brain_area_lfp']: brain area names for the LFP channels.
*   dat['trough_to_peak']: measures the width of the action potential waveform for each neuron. Widths <=10 samples are "putative fast spiking neurons".
*   dat['waveform_w']: temporal components of spike waveforms. w@u reconstructs the time by channels action potential shape.
*   dat['waveform_u]: spatial components of spike waveforms.
*   dat['%X%_passive']: same as above for X = {spks, lfp, pupil, wheel, contrast_left, contrast_right} but for passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses.

So the variables we need for the behavioral analysis are:
'mouse_name', 'date_exp', 'contrast_right', 'contrast_left', 'gocue', 'response_time','response', 'feedback_time', 'feedback_type'


gocue, response_time, feedback_time are all arrays of arrays (an array of 1 cell arrays) and refuse to be tranformed to a df straightforwardly, so we're going to convert them in this really stupid ghetto way, then add them back. Sorry for anyone looking at this.

In [23]:
count=0
for i in range(0,len(dat["active_trials"])):
  if (dat["active_trials"][i]==True):
    count=count+1
  else:
    count=count

print(count)
print(len(dat["active_trials"]))
print(len(dat['response_time']))
      

340
450
340


In [58]:
behav_dat = pd.DataFrame()
cum_trial_num=0

for k in range(len(alldat)):
  temp=alldat[k]
  t1=temp['response_time']
  t2=temp['gocue']
  t3=temp['feedback_time']

  response_time, gocue, feedback_time=np.zeros(len(t1)),np.zeros(len(t2)),np.zeros(len(t3))
  fb=np.zeros(len(t4))
  trial, session=np.zeros(len(t1)),np.zeros(len(t1))
  prev_rew, avg_rew_hist_3 =np.zeros(len(t1)),np.zeros(len(t1))
  cum_trial_num=cum_trial_num+len(t1)

  for i in range(0,len(t1)):
    response_time[i]=t1[i][0]
    gocue[i]=t2[i][0]
    feedback_time=t3[i][0]
    trial[i]=i
    session[i]=k
    if (i==0):
      prev_rew[i]=0
      cum_rew_rate[i]
    else:
      if (temp['feedback_type'][i-1]==1):
        prev_rew[i]=1
      else:
        prev_rew[i]=0
    #cum_rew_rate[i]=np.mean(cum_rew_rate[0:i])
    i+=1

  #avg_rew_hist_3 = []
  window_size=3
  while i < len(prev_rew) - window_size + 1:
    this_window = prev_rew[i : i + window_size]

    window_average = sum(this_window) / window_size
    avg_rew_hist_3.append(window_average)
    i += 1


  your_keys=['mouse_name', 'date_exp', 'contrast_right', 'contrast_left',  'response', 'response_time', 'feedback_type']
  sess_behav = { your_key: temp[your_key] for your_key in your_keys }
  sess_behav["response_time"]= response_time 
  sess_behav["gocue"]=gocue
  sess_behav["feedback_time"]=feedback_time
  sess_behav["trial_num"]=trial
  sess_behav["session"]=session
  sess_behav["prev_reward"]=prev_rew
  sess_behav["rew_hist_3"]=avg_rew_hist_3


  sess_behav_long =pd.DataFrame.from_dict(sess_behav)
  
  if k==0:
    behav_dat=sess_behav_long
  else:
    behav_dat=pd.concat([behav_dat, sess_behav_long], ignore_index=True)
  


behav_dat.tail()

,mouse_name,date_exp,contrast_right,contrast_left,response,response_time,feedback_type,gocue,feedback_time,trial_num,session,prev_reward,rew_hist_3
10045,Theiler,2017-10-11,0.25,1.0,0.0,2.297503,-1.0,0.794097,2.101029,338.0,38.0,0.0,0.0
10046,Theiler,2017-10-11,0.25,1.0,-1.0,1.158803,-1.0,0.524700,2.101029,339.0,38.0,0.0,0.0
10047,Theiler,2017-10-11,0.25,1.0,0.0,2.003709,-1.0,0.504257,2.101029,340.0,38.0,0.0,0.0
10048,Theiler,2017-10-11,0.25,1.0,0.0,2.076758,-1.0,0.574262,2.101029,341.0,38.0,0.0,0.0
10049,Theiler,2017-10-11,0.25,1.0,0.0,2.101029,-1.0,0.589720,2.101029,342.0,38.0,0.0,0.0


Awesome! Now we need to calculate the following variables:

*   Trial type (no-go, two image, one image, equal contrast)
  * no_go if contrast is 0 on both sides
  * one_image
  * two_image_unequal: two images presented and one higher than the other
  * two_image_equal: two images with equal contrast
*   Accuracy (correct=0, incorrect=1)
  * no_go trials are only correct if 0
  * two_image_equal trials are correct if -1 or 1
  * one_image and two_image_unequal trials are correct depending on the direction
* Difference in contrast stim
  * 0 if no go or same contrast level
  * Negative (<0) if right side has more contrast
  * Positive (>0) if left side has more contrast 

*  Feedback type corresponds with correctness (better than expected, as expected, worse than expected)






In [59]:
##creating trial type variable
def conditions(s):
    if (s['contrast_right']==0 and s['contrast_left']==0):
        return "no_go"
    elif (s['contrast_right']==0 and s['contrast_left']>0):
        return "one_image"
    elif (s['contrast_left']==0 and s['contrast_right']>0):
        return "one_image"
    elif (s['contrast_left']!=0 and s['contrast_right']!=0):
        if (s['contrast_left']==s['contrast_right']):
          return "two_image_equal"
        else:
          return "two_image_unequal"
    else:
      return "???"

behav_dat['trial_type'] = behav_dat.apply(conditions, axis=1)

behav_dat.head()

pd.crosstab(behav_dat['trial_type'], [behav_dat['contrast_right'], behav_dat['contrast_left']])

contrast_right     0.00                0.25       ... 0.50      1.00               
contrast_left      0.00 0.25 0.50 1.00 0.00 0.25  ... 0.50 1.00 0.00 0.25 0.50 1.00
trial_type                                        ...                              
no_go              2649    0    0    0    0    0  ...    0    0    0    0    0    0
one_image             0  323  737  944  348    0  ...    0    0  945    0    0    0
two_image_equal       0    0    0    0    0  225  ...  226    0    0    0    0  221
two_image_unequal     0    0    0    0    0    0  ...    0  322    0  670  295    0

[4 rows x 16 columns]

In [60]:
##creating accuracy variable

def corr_cond(d):
  if (d['trial_type'=='no_go']):
    if (d['response']==0):
      return 1
    else:
      return 0
  elif (d['trial_type'=='two_image_equal']):
    if (d['response']!=0):
      return 1
    else:
      return 0
  elif (d['trial_type'=='two_image_unequal'] or d['trial_type'=='one_image']):
    if (d['contrast_right']>d['contrast_left']):
      if (d['response']==-1) :
        return 1
      else:
        return 0
    elif (d['contrast_right']<d['contrast_left']):
      if (d['response']==1):
        return 1
      else:
        return 0
  else:
    return 2 #just as a catch

behav_dat['accuracy'] = behav_dat.apply(corr_cond, axis=1)

behav_dat.head()



,mouse_name,date_exp,contrast_right,contrast_left,response,response_time,feedback_type,gocue,feedback_time,trial_num,session,prev_reward,rew_hist_3,trial_type,accuracy
0,Cori,2016-12-14,0.0,1.0,1.0,1.150204,1.0,1.027216,1.474423,0.0,0.0,0.0,0.0,one_image,0
1,Cori,2016-12-14,0.5,0.0,-1.0,1.399503,1.0,0.874414,1.474423,1.0,0.0,1.0,0.0,one_image,0
2,Cori,2016-12-14,0.5,1.0,1.0,0.949291,1.0,0.825213,1.474423,2.0,0.0,1.0,0.0,two_image_unequal,0
3,Cori,2016-12-14,0.0,0.0,0.0,2.266802,1.0,0.761612,1.474423,3.0,0.0,1.0,0.0,no_go,1
4,Cori,2016-12-14,1.0,0.5,1.0,0.816776,-1.0,0.662010,1.474423,4.0,0.0,1.0,0.0,two_image_unequal,0


In [61]:
pd.crosstab(behav_dat['trial_type'], behav_dat['accuracy'])  #ok I mean it  looks like I sorted this correctly but it's really a pretty low accuracy overall


accuracy,0,1
trial_type,,
no_go,881,1768
one_image,3087,898
two_image_equal,558,114
two_image_unequal,2219,525


In [62]:
behav_dat['overall_accuracy'] = behav_dat.groupby(['session'])['accuracy'].transform(np.mean)
pd.crosstab(behav_dat['overall_accuracy'],behav_dat['mouse_name'])  #ok Lederberg has pretty pitiful performance

mouse_name,Cori,Forssmann,Hench,Lederberg,Moniz,Muller,Radnitz,Richards,Tatum,Theiler
overall_accuracy,,,,,,,,,,
0.186667,0,0,0,300,0,0,0,0,0,0
0.188235,0,0,0,340,0,0,0,0,0,0
0.227612,0,0,0,268,0,0,0,0,0,0
0.232143,0,0,0,224,0,0,0,0,0,0
0.240099,0,0,0,404,0,0,0,0,0,0
0.240506,0,0,0,316,0,0,0,0,0,0
0.246032,0,252,0,0,0,0,0,0,0,0
0.254019,0,0,0,0,0,0,0,0,311,0
0.259009,0,0,0,0,0,444,0,0,0,0


In [63]:
#contrast difference variable

behav_dat['contrast_diff'] = behav_dat["contrast_left"]-behav_dat["contrast_right"]
behav_dat.head()

#from dfply import *

#(behav_dat >>
#  group_by(trial_type) >>
#  summarize(accuracy = accuracy.mean())
#)


,mouse_name,date_exp,contrast_right,contrast_left,response,response_time,feedback_type,gocue,feedback_time,trial_num,session,prev_reward,rew_hist_3,trial_type,accuracy,overall_accuracy,contrast_diff
0,Cori,2016-12-14,0.0,1.0,1.0,1.150204,1.0,1.027216,1.474423,0.0,0.0,0.0,0.0,one_image,0,0.345794,1.0
1,Cori,2016-12-14,0.5,0.0,-1.0,1.399503,1.0,0.874414,1.474423,1.0,0.0,1.0,0.0,one_image,0,0.345794,-0.5
2,Cori,2016-12-14,0.5,1.0,1.0,0.949291,1.0,0.825213,1.474423,2.0,0.0,1.0,0.0,two_image_unequal,0,0.345794,0.5
3,Cori,2016-12-14,0.0,0.0,0.0,2.266802,1.0,0.761612,1.474423,3.0,0.0,1.0,0.0,no_go,1,0.345794,0.0
4,Cori,2016-12-14,1.0,0.5,1.0,0.816776,-1.0,0.662010,1.474423,4.0,0.0,1.0,0.0,two_image_unequal,0,0.345794,-0.5
